In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('rows.csv')

df1 = df[['Product', 'Consumer complaint narrative', 'Company', 'State', 'Tags', 'Submitted via', 'Timely response?']].copy()

C:\Users\nickn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4,5,6,11,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# Columns are show in rows for easy reading
df1.head(2).T

,0,1
Product,Checking or savings account,Checking or savings account
Consumer complaint narrative,NaN,NaN
Company,NAVY FEDERAL CREDIT UNION,BOEING EMPLOYEES CREDIT UNION
State,FL,WA
Tags,Older American,NaN
Submitted via,Web,Referral
Timely response?,Yes,Yes


In [4]:
# Removing missing values (NaN)
df1 = df1[pd.notnull(df1['Consumer complaint narrative'])]
df1 = df1[pd.notnull(df1['Tags'])]
df1 = df1[pd.notnull(df1['State'])]

In [5]:
# Renaming categories
df1.replace({'Product':
             {'Credit reporting, credit repair services, or other personal consumer reports':
              'Credit reporting, repair, or other',
              'Credit reporting': 'Credit reporting, repair, or other',
             'Credit card': 'Credit card or prepaid card',
             'Prepaid card': 'Credit card or prepaid card',
             'Payday loan': 'Payday loan, title loan, or personal loan',
             'Money transfer': 'Money transfer, virtual currency, or money service',
             'Virtual currency': 'Money transfer, virtual currency, or money service'}},
            inplace= True)
# Convert company to uppercase
df1['Company'] = df1['Company'].str.upper()
# df1['category_id'] = df1['Product'].factorize()[0]
# print(pd.DataFrame(df1['Company'].unique()).values)

In [6]:
# Vectorize company and consumer narrative
# df1['Consumer complaint narrative'] = hero.tfidf(df1['Consumer complaint narrative'])
# df1['Company'] = hero.tfidf(df1['Company'])

In [7]:
# Factorize Products, state, tags, timely response
df1['Product'] = df1['Product'].factorize()[0].astype(np.float32)
df1['State'] = df1['State'].factorize()[0].astype(np.float32)
df1['Tags'] = df1['Tags'].factorize()[0].astype(np.float32)
df1['Tags'] = df1['Tags'].factorize()[0].astype(np.float32)
df1['Submitted via'] = df1['Submitted via'].factorize()[0].astype(np.float32)
df1['Timely response?'] = df1['Timely response?'].factorize()[0].astype(np.float32)
df1['Company'] = df1['Company'].factorize()[0].astype(np.float32)
df1['Consumer complaint narrative'] = df1['Consumer complaint narrative'].factorize()[0].astype(np.float32)
df1.head()

,Product,Consumer complaint narrative,Company,State,Tags,Submitted via,Timely response?
31078,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35401,0.0,1.0,0.0,1.0,0.0,0.0,0.0
36277,0.0,2.0,1.0,2.0,0.0,0.0,0.0
36566,0.0,3.0,0.0,2.0,0.0,0.0,0.0
36687,0.0,4.0,0.0,3.0,0.0,0.0,0.0


In [8]:
# Get independent variables
x = df1.iloc[:, :-1].values
# Get dependent variables
y = df1.iloc[:, -1].values
print(x)

[[0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 1.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 2.0000e+00 1.0000e+00 2.0000e+00 0.0000e+00 0.0000e+00]
 ...
 [0.0000e+00 6.4240e+04 1.5000e+01 4.0000e+00 1.0000e+00 0.0000e+00]
 [2.0000e+00 6.4241e+04 1.4860e+03 5.0000e+00 1.0000e+00 0.0000e+00]
 [0.0000e+00 6.4242e+04 1.0000e+00 2.1000e+01 2.0000e+00 0.0000e+00]]


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
# vectorizer.fit(x_train.ravel())
# x_train = vectorizer.transform(x_train)
# x_test  = vectorizer.transform(x_test)

In [10]:
from keras.models import Sequential
from keras.layers import Dense
from keras.backend import clear_session



input_dim = x_train.shape[1]
# Define the keras model
model = Sequential()
model.add(Dense(12, input_dim=input_dim, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                84        
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 197
Trainable params: 197
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
# Compile or execute model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [12]:
# Train model on training sample
print('Training results')
# Clear previous
clear_session()
trained_model = model.fit(x_train, y_train, epochs=100, batch_size=10)

Training results
Epoch 1/100
5284/5284 [==============================] - 5s 1ms/step - loss: 4.9379 - accuracy: 0.9382
Epoch 2/100
5284/5284 [==============================] - 4s 737us/step - loss: 1.9028 - accuracy: 0.9446
Epoch 3/100
5284/5284 [==============================] - 4s 707us/step - loss: 1.3739 - accuracy: 0.9468
Epoch 4/100
5284/5284 [==============================] - 4s 724us/step - loss: 0.9392 - accuracy: 0.9500
Epoch 5/100
5284/5284 [==============================] - 4s 718us/step - loss: 0.6155 - accuracy: 0.9534
Epoch 6/100
5284/5284 [==============================] - 5s 864us/step - loss: 0.3324 - accuracy: 0.9614
Epoch 7/100
5284/5284 [==============================] - 4s 776us/step - loss: 0.1644 - accuracy: 0.9686
Epoch 8/100
5284/5284 [==============================] - 4s 718us/step - loss: 0.1344 - accuracy: 0.9705
Epoch 9/100
5284/5284 [==============================] - 4s 700us/step - loss: 0.1431 - accuracy: 0.9711
Epoch 10/100
5284/5284 [================

5284/5284 [==============================] - 4s 717us/step - loss: 0.1217 - accuracy: 0.9709
Epoch 78/100
5284/5284 [==============================] - 4s 669us/step - loss: 0.1129 - accuracy: 0.9708
Epoch 79/100
5284/5284 [==============================] - 4s 671us/step - loss: 0.1200 - accuracy: 0.9710
Epoch 80/100
5284/5284 [==============================] - 4s 671us/step - loss: 0.1148 - accuracy: 0.9708
Epoch 81/100
5284/5284 [==============================] - 4s 717us/step - loss: 0.1119 - accuracy: 0.9708
Epoch 82/100
5284/5284 [==============================] - 4s 842us/step - loss: 0.1157 - accuracy: 0.9707
Epoch 83/100
5284/5284 [==============================] - 4s 696us/step - loss: 0.1119 - accuracy: 0.9709
Epoch 84/100
5284/5284 [==============================] - 4s 710us/step - loss: 0.1119 - accuracy: 0.9708
Epoch 85/100
5284/5284 [==============================] - 4s 803us/step - loss: 0.1140 - accuracy: 0.9704
Epoch 86/100
5284/5284 [==============================] - 5

In [13]:
loss, accuracy = model.evaluate(x_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))

loss, accuracy = model.evaluate(x_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9711
Testing Accuracy:  0.9743


In [14]:
from ann_visualizer.visualize import ann_viz
ann_viz(model, view=True, filename='network.gv', title='Neural Network')